###### References: 
- https://docs.python.org/3/library/asyncio.html
- Fluent Python, 2nd Edition, by Luciano Ramalho. Chapter 22: Dynamic Attributes and Properties

### Part  V. Metaprogramming


In [1]:
import json

In [2]:
with open('osconfeed.json') as fp:
    feed = json.load(fp)
    
sorted(feed['Schedule'].keys())

['conferences', 'events', 'speakers', 'venues']

In [3]:
for key, value in sorted(feed['Schedule'].items()):
    print(f'{len(value):3} {key}')

  1 conferences
484 events
357 speakers
 53 venues


In [4]:
feed['Schedule']['speakers'][-1]['name']

'Carina C. Zona'

In [5]:
feed['Schedule']['speakers'][-1]['serial']

141590

In [6]:
feed['Schedule']['events'][40]['name']

'There *Will* Be Bugs'

In [7]:
feed['Schedule']['events'][40]['speakers']

[3471, 5199]

## Exploring JSON-Like Data with Dynamic Attributes

In [8]:
from collections import abc


class FrozenJSON:
    """A read-only façade for navigating a JSON-like object
       using attribute notation
    """

    def __init__(self, mapping):
        self.__data = dict(mapping)  # build a dict

    def __getattr__(self, name):  # called only if attribute does not exist
        try:
            return getattr(self.__data, name)  # return matching name
        except AttributeError:
            return FrozenJSON.build(self.__data[name])  # otherwise fetch item from key name form built dict

    def __dir__(self):  # to support built in dir()
        return self.__data.keys()

    @classmethod
    def build(cls, obj):  # alternate constuctor
        if isinstance(obj, abc.Mapping):  # build a FrozenJSON if object is a mapping
            return cls(obj)
        elif isinstance(obj, abc.MutableSequence):  # build a list if is Multable Sequence
            return [cls.build(item) for item in obj]
        else:  # otherwise return as it is
            return obj

In [9]:
raw_feed = json.load(open('osconfeed.json'))
feed = FrozenJSON(raw_feed) 
len(feed.Schedule.speakers)

357

In [10]:
feed.keys()

dict_keys(['Schedule'])

In [11]:
sorted(feed.Schedule.keys())

['conferences', 'events', 'speakers', 'venues']

In [12]:
for key, value in sorted(feed.Schedule.items()):
    print(f'{len(value):3} {key}')

  1 conferences
484 events
357 speakers
 53 venues


In [13]:
feed.Schedule.speakers[-1].name

'Carina C. Zona'

In [14]:
talk = feed.Schedule.events[40]
type(talk)

__main__.FrozenJSON

In [15]:
talk.name

'There *Will* Be Bugs'

In [16]:
talk.speakers

[3471, 5199]

In [17]:
talk.flavour

KeyError: 'flavour'

## The Invalid Attribute Name Problem

In [18]:
student = FrozenJSON({'name': 'Jim Bo', 'class': 1982})

In [19]:
student.class

SyntaxError: invalid syntax (144001523.py, line 1)

In [20]:
getattr(student, 'class')

1982

In [21]:
import keyword

In [22]:
class FrozenJSON:
    """A read-only façade for navigating a JSON-like object
       using attribute notation
    """

    def __init__(self, mapping):
        self.__data = {}
        for key, value in mapping.items():
            if keyword.iskeyword(key): 
                key += '_'
            self.__data[key] = value

    def __getattr__(self, name):
        try:
            return getattr(self.__data, name)
        except AttributeError:
            return FrozenJSON.build(self.__data[name])

    def __dir__(self):  # <5>
        return self.__data.keys()

    @classmethod
    def build(cls, obj):
        if isinstance(obj, abc.Mapping):
            return cls(obj)
        elif isinstance(obj, abc.MutableSequence):
            return [cls.build(item) for item in obj]
        else:  # <8>
            return obj

In [23]:
student = FrozenJSON({'name': 'Jim Bo', 'class': 1982})

In [24]:
student.class_

1982

## Flexible Object Creation with __new__

In [25]:
class FrozenJSON:
    """A read-only façade for navigating a JSON-like object
       using attribute notation
    """

    def __new__(cls, arg):  # a class method
        if isinstance(arg, abc.Mapping):
            return super().__new__(cls)  # calling from obj base class
        elif isinstance(arg, abc.MutableSequence):  # exactly as in the old build method
            return [cls(item) for item in arg]
        else:
            return arg

    def __init__(self, mapping):
        self.__data = {}
        for key, value in mapping.items():
            if keyword.iskeyword(key):
                key += '_'
            self.__data[key] = value

    def __getattr__(self, name):
        try:
            return getattr(self.__data, name)
        except AttributeError:
            return FrozenJSON(self.__data[name])  # call the class instead of .build

    def __dir__(self):
        return self.__data.keys()

# Computed Properties
## Step 1: Data-Driven Attribute Creation

In [26]:
JSON_PATH = 'osconfeed.json'

class Record:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)  # shortcut to build an instance with attr from keyword arg

    def __repr__(self):
        return f'<{self.__class__.__name__} serial={self.serial!r}>'  # use serial field to build the custom Record

def load(path=JSON_PATH):
    records = {}  # load will return a dict of record instances
    with open(path) as fp:
        raw_data = json.load(fp)  
    for collection, raw_records in raw_data['Schedule'].items():  # iterate over the four top levels
        record_type = collection[:-1]  # list names without the last character (s)
        for raw_record in raw_records:
            key = f'{record_type}.{raw_record["serial"]}' # build the in the format 'speaker.3741'
            records[key] = Record(**raw_record)  # create an instance and save with key
    return records

In [27]:
records = load(JSON_PATH)

In [28]:
speaker = records['speaker.3471']
speaker

<Record serial=3471>

In [29]:
speaker.name, speaker.twitter

('Anna Martelli Ravenscroft', 'annaraven')

## Step 2: Property to Retrieve a Linked Record

In [30]:
import inspect

In [31]:
class Record:

    __index = None  # to hold reference to the dict returned by load

    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

    def __repr__(self):
        return f'<{self.__class__.__name__} serial={self.serial!r}>'

    @staticmethod  # to explicitly state that it is not dynamic
    def fetch(key):
        if Record.__index is None:  # populate the index if neededd.
            Record.__index = load()
        return Record.__index[key]  # use it to retrieve the recordd with the given key

    
class Event(Record):  # Extends record

    def __repr__(self):
        try:
            return f'<{self.__class__.__name__} {self.name!r}>'  # the name attr is used to produce a custom repr
        except AttributeError:
            return super().__repr__()

    @property
    def venue(self):
        key = f'venue.{self.venue_serial}'
        return self.__class__.fetch(key) # the venue property builds a key from the venue_serial attr, and passes it to the fetch class method

In [32]:
def load(path=JSON_PATH):
    records = {}
    with open(path) as fp:
        raw_data = json.load(fp)
    for collection, raw_records in raw_data['Schedule'].items():
        record_type = collection[:-1] 
        cls_name = record_type.capitalize()  # Capitalize the record_type to get a possible class name
        cls = globals().get(cls_name, Record)  # get the obj from global scope
        if inspect.isclass(cls) and issubclass(cls, Record):  
            factory = cls  # bind the factory name to it if it is a subclass of Record.
        else:
            factory = Record  # otherwise bind factory name to Record.
        for raw_record in raw_records:  # creates  the key and saves the record
            key = f'{record_type}.{raw_record["serial"]}'
            records[key] = factory(**raw_record)  # the obj stored is constructed by factory, depending on type.
    return 

## Step 3: Property Overriding an Existing Attribute

In [33]:
class Event(Record):

    def __repr__(self):
        try:
            return f'<{self.__class__.__name__} {self.name!r}>'
        except AttributeError:
            return super().__repr__()

    @property
    def venue(self):
        key = f'venue.{self.venue_serial}'
        return self.__class__.fetch(key)

    @property
    def speakers(self):
        spkr_serials = self.__dict__['speakers']  # retrieve directly from the instance dict
        fetch = self.__class__.fetch
        return [fetch(f'speaker.{key}')
                for key in spkr_serials]

## Step 4: Bespoke Property Cache

In [34]:
class Event(Record):

    def __init__(self, **kwargs):
        self.__speaker_objs = None
        super().__init__(**kwargs)

    def __repr__(self):
        try:
            return f'<{self.__class__.__name__} {self.name!r}>'
        except AttributeError:
            return super().__repr__()

    @property
    def venue(self):
        key = f'venue.{self.venue_serial}'
        return self.__class__.fetch(key)

    @property
    def speakers(self):
        if self.__speaker_objs is None: # the objs are initialized to none, and checking as a simple caching technique
            spkr_serials = self.__dict__['speakers']
            fetch = self.__class__.fetch
            self.__speaker_objs = [fetch(f'speaker.{key}')
                    for key in spkr_serials]
        return self.__speaker_objs

## Step 5:  Caching Properties with `functools`

`@cached_property` decorator does not create a full-fledgeed property, it creates a _nonoverriding descripitor_

A descriptor is an object that manages the access to an attribute in another class.

In [35]:
from functools import cached_property, cache

class Event(Record):

    def __repr__(self):
        try:
            return f'<{self.__class__.__name__} {self.name!r}>'
        except AttributeError:
            return super().__repr__()

    @cached_property
    def venue(self):
        key = f'venue.{self.venue_serial}'
        return self.__class__.fetch(key)

    @property  
    @cache  
    def speakers(self):
        spkr_serials = self.__dict__['speakers']
        fetch = self.__class__.fetch
        return [fetch(f'speaker.{key}')
                for key in spkr_serials]

In [36]:
class Record:

    __index = None

    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

    def __repr__(self):
        return f'<{self.__class__.__name__} serial={self.serial!r}>'

    @staticmethod
    def fetch(key):
        if Record.__index is None:
            Record.__index = load()
        return Record.__index[key]


class Event(Record):

    def __repr__(self):
        try:
            return f'<{self.__class__.__name__} {self.name!r}>'
        except AttributeError:
            return super().__repr__()

# tag::SCHEDULE5_CACHED_PROPERTY[]
    @cached_property
    def venue(self):
        key = f'venue.{self.venue_serial}'
        return self.__class__.fetch(key)
# end::SCHEDULE5_CACHED_PROPERTY[]
# tag::SCHEDULE5_PROPERTY_OVER_CACHE[]
    @property  # <1>
    @cache  # <2>
    def speakers(self):
        spkr_serials = self.__dict__['speakers']
        fetch = self.__class__.fetch
        return [fetch(f'speaker.{key}')
                for key in spkr_serials]
# end::SCHEDULE5_PROPERTY_OVER_CACHE[]

def load(path=JSON_PATH):
    records = {}
    with open(path) as fp:
        raw_data = json.load(fp)
    for collection, raw_records in raw_data['Schedule'].items():
        record_type = collection[:-1]
        cls_name = record_type.capitalize()
        cls = globals().get(cls_name, Record)
        if inspect.isclass(cls) and issubclass(cls, Record):
            factory = cls
        else:
            factory = Record
        for raw_record in raw_records:
            key = f'{record_type}.{raw_record["serial"]}'
            records[key] = factory(**raw_record)
    return records

In [37]:
records = load(JSON_PATH)

In [38]:
event = records['event.33950']

In [39]:
event

<Event 'There *Will* Be Bugs'>

In [40]:
event.venue

<Record serial=1449>

In [41]:
event.venue.name

'Portland 251'

In [42]:
for spkr in event.speakers:
    print(f'{spkr.serial}: {spkr.name}')

3471: Anna Martelli Ravenscroft
5199: Alex Martelli


# Using Property for Attribute Validation

## LineItem  #1: Class  for an  Item in an  Order

In [43]:
class LineItem:

    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price

    def subtotal(self):
        return self.weight * self.price

In [44]:
raisins = LineItem('Golden raisins', 10, 6.95)

In [45]:
raisins.subtotal()

69.5

In [46]:
raisins.weight  = -20

In [47]:
raisins.subtotal()

-139.0

## LineItem Take #2:  A  Validating  Property

In [48]:
class LineItem:

    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight 
        self.price = price

    def subtotal(self):
        return self.weight * self.price

    @property  # decorates the getter method
    def weight(self):  
        return self.__weight  # actual value stored in private attribute

    @weight.setter  # this ties the getter and setter together
    def weight(self, value):
        if value > 0:
            self.__weight = value 
        else:
            raise ValueError('value must be > 0') 

In [49]:
walnuts = LineItem('walnuts', 0, 10.00)

ValueError: value must be > 0

# A Proper Look at Properties

## Classic implementation:

In [50]:
class LineItem:

    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price

    def subtotal(self):
        return self.weight * self.price

    def get_weight(self):  
        return self.__weight

    def set_weight(self, value):
        if value > 0:
            self.__weight = value
        else:
            raise ValueError('value must be > 0')

    weight = property(get_weight, set_weight) 

## Properties Overridde Instance Attributes

In [51]:
class Class:
    data = 'the class data attr'
    @property
    def prop(self):
        return 'the prop value'

In [52]:
obj = Class()

In [53]:
vars(obj)

{}

In [54]:
obj.data

'the class data attr'

In [55]:
obj.data = 'bar'

In [56]:
vars(obj)

{'data': 'bar'}

In [57]:
obj.data

'bar'

In [58]:
Class.data

'the class data attr'

In [59]:
Class.prop

In [60]:
obj.prop

'the prop value'

In [61]:
obj.prop = 'foo'

AttributeError: can't set attribute 'prop'

In [62]:
obj.__dict__['prop']  = 'foo'

In [63]:
vars(obj)

{'data': 'bar', 'prop': 'foo'}

In [64]:
obj.prop

'the prop value'

In [65]:
Class.prop  = 'barz' #  Class.prop destroys the property object
obj.prop

'foo'

In [66]:
# instance attribute
obj.data

'bar'

In [67]:
# class attribute
Class.data

'the class data attr'

In [68]:
# overwrite the Class.data with new property
Class.data = property(lambda self: 'the "data" prop value')

In [69]:
obj.data

'the "data" prop value'

In [70]:
del Class.data

In [71]:
obj.data

'bar'

# Property Documentation

In [72]:
class Foo:
    @property
    def bar(self):
        '''The bar attribute'''
        return self.__dict__['bar']
    
    @bar.setter
    def bar(self, value):
        self.__dict__['bar'] = value

In [73]:
help(Foo)

Help on class Foo in module __main__:

class Foo(builtins.object)
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  bar
 |      The bar attribute



# Coding a Property Factory

In [74]:
def quantity(storage_name):  # <1>

    def qty_getter(instance):  # <2>
        return instance.__dict__[storage_name]  # <3>

    def qty_setter(instance, value):  # <4>
        if value > 0:
            instance.__dict__[storage_name] = value  # <5>
        else:
            raise ValueError('value must be > 0')

    return property(qty_getter, qty_setter)

In [75]:
class LineItem:
    weight = quantity('weight')  # <1>
    price = quantity('price')  # <2>

    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight  # <3>
        self.price = price

    def subtotal(self):
        return self.weight * self.price

In [76]:
nutmeg = LineItem('Moluccan nutmeg', 8, 13.95)

In [77]:
nutmeg.weight, nutmeg.price

(8, 13.95)

In [78]:
sorted(vars(nutmeg).items())

[('description', 'Moluccan nutmeg'), ('price', 13.95), ('weight', 8)]

In [79]:
nutmeg.__dict__

{'description': 'Moluccan nutmeg', 'weight': 8, 'price': 13.95}

In [80]:
nutmeg.subtotal()

111.6

# Handling Attribute Deletion

In [81]:
class Demo:
    pass

In [82]:
d = Demo()

In [83]:
d.color = 'green'

In [84]:
d.color

'green'

In [85]:
del d.color

In [86]:
d.color

AttributeError: 'Demo' object has no attribute 'color'

In [87]:
class BlackKnight:

    def __init__(self):
        self.members = ['an arm', 'another arm',
                        'a leg', 'another leg']
        self.phrases = ["'Tis but a scratch.",
                        "It's just a flesh wound.",
                        "I'm invincible!",
                        "All right, we'll call it a draw."]

    @property
    def member(self):
        print('next member is:')
        return self.members[0]

    @member.deleter
    def member(self):
        text = 'BLACK KNIGHT (loses {})\n-- {}'
        print(text.format(self.members.pop(0), self.phrases.pop(0)))

In [88]:
knight = BlackKnight()

In [89]:
knight.member

next member is:


'an arm'

In [90]:
del knight.member

BLACK KNIGHT (loses an arm)
-- 'Tis but a scratch.


In [91]:
del knight.member

BLACK KNIGHT (loses another arm)
-- It's just a flesh wound.


In [92]:
del knight.member

BLACK KNIGHT (loses a leg)
-- I'm invincible!


In [93]:
del knight.member

BLACK KNIGHT (loses another leg)
-- All right, we'll call it a draw.
